In [3]:
import folium
import pandas as pd
import geopandas as gpd
import json
from folium.plugins import MeasureControl
from folium.plugins import Search
from folium.plugins import LocateControl
from folium.features import CustomIcon
from folium.plugins import MarkerCluster
from folium.plugins import Draw
from folium import IFrame
from folium import plugins

# Crea un mapa base con un centro y un nivel de zoom determinados
m = folium.Map(location=[20.5153, -98.7459], zoom_start=9, tiles="openstreetmap", min_zoom=8, control_scale=True,max_bounds=[[21.7010, -100.0662], [19.3611,  -97.1548]])
# Se agregan los mapa base
folium.TileLayer(
    tiles='stamenterrain',
    attr='',
    name='Terrreno'
).add_to(m)

folium.TileLayer(
    tiles='cartodbdark_matter',
    name='Obscuro'
).add_to(m)
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(m)

# Agrego capas convirtiendo los geojson en Geodataframes
psdh_gdf=gpd.read_file("psdh.geojson")
h_huasteca_gdf=gpd.read_file('h_huasteca1.geojson')
hospitales_gdf=gpd.read_file('hospitales.geojson')
salud_gdf=gpd.read_file('salud.geojson')
#ageb_csv= pd.read_csv("ageb.csv")
# Creo un estilo 
bordersStyle = {'color': 'brown',
                'weight': 1,
                'fillColor': '#DDc9A3',
                'opacity':1,
                'line_opacity': .5,
               }

hosp_legend_html = '''
     <div style="width: 200px; height: 120px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color: rgba(255, 255, 255, 0.7); position:sticky; top:10px;
                 backdrop-filter: blur(5px); border-radius: 10px; padding: 10px;
                ">
        <div style="display: flex; justify-content: center;">
            <h4 style="margin: 0;"><b>Hospitales públicos</b></h4>
        </div><br>
        <div style="display: flex; margin-top: 5px;">
            <div style="width: 20px; height: 20px; margin-right: 5px; background-color: #51d1f6; border-radius: 50%;"></div>
            <p style="margin: 0;">En funcionamiento</p>
        </div>
        <div style="display: flex; margin-top: 5px;">
            <div style="width: 20px; height: 20px; margin-right: 5px; background-color: #008f39; border-radius: 50%;"></div>
            <p style="margin: 0;">Próximo a construir</p>
        </div>         
     </div>
'''

hosp_leg=folium.map.Marker(
    [20.6073,  -97.4100],
    icon=folium.features.DivIcon(html=hosp_legend_html),
    draggable=True
)


#Popups
popup_huejutla = folium.Popup(
    '<div style="background-color: #FFF; border-radius: 20px; padding: 0px; width: 450px; height: 900px;"> \
       <b><h3 style="text-align: center;color: white;background-color: #691c32">Construcción del Nuevo Hospital Regional de Huejutla</h3></b> \
        <iframe src="https://criteriohidalgo.com/regiones/proyectan-nuevo-hospital-regional-en-huejutla" style="border:none; width:100%; height:80%;"> </iframe>\
            Más información \
        </a> \
    </div>',
    max_width=450   
)
psdh=folium.GeoJson(psdh_gdf,
               name='% Población sin derechohabiencia',
               tooltip=folium.GeoJsonTooltip(fields=["NOMGEO", "POR_PSD"],aliases=["Municipio: ","% Población sin derechohabiencia: "], sticky=True, localize=True),
               style_function=lambda x: bordersStyle,
              ).add_to(m)

psd1=folium.Choropleth(
    geo_data=psdh_gdf,
    data=psdh_gdf,
    name='% Población sin derechohabiencia',
    columns=['CVEGEO', 'POR_PSD'],
    key_on='feature.properties.CVEGEO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='% de Población sin derechohabiencia',
    show=True,
    control=True,
    highlight=True,
    smooth_factor=True
).add_to(m)

salud1=folium.Choropleth(
    geo_data=salud_gdf,
    data=salud_gdf,
    name='% Personas con discapacidad motríz',
    columns=['CVEGEO', '% Personas con discapacidad motriz'],
    key_on='feature.properties.CVEGEO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='% Personas con discapacidad motríz',
    show=False,
    control=True,
    highlight=True,
    smooth_factor=True
).add_to(m)

salud2=folium.Choropleth(
    geo_data=salud_gdf,
    data=salud_gdf,
    name='% Personas con alguna condición mental',
    columns=['CVEGEO', '%Personas con alguna condición mental'],
    key_on='feature.properties.CVEGEO',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='%Personas con alguna condición mental',
    show=False,
    control=True,
    highlight=True,
    smooth_factor=True
).add_to(m)

hospitales=folium.GeoJson(hospitales_gdf,
                        name='Hospitales públicos',
               tooltip=folium.GeoJsonTooltip(fields=["Name"],aliases=["Nombre: "], sticky=False, localize=True),
                       zoom_on_click=True, show=True,
                       marker=folium.Marker(icon=folium.Icon(color='blue',icon='fa-hospital-o',prefix="fa"))).add_to(m)

h_huasteca = folium.GeoJson(h_huasteca_gdf,
                        name='Hospital Regional de Huejutla (Próximo a construir)',
                            popup=popup_huejutla,
               tooltip=folium.GeoJsonTooltip(fields=["Nombre"],aliases=["Próximo a construir: "], sticky=False, localize=True),
                       zoom_on_click=True, show=True,
                       marker=folium.Marker(icon=folium.Icon(color='green',icon='fa-hospital-o',prefix="fa"))).add_to(m)


#Se añade la leyenda html a la capa hospitales
hospitales.add_child(hosp_leg)


psd1.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=["NOMGEO", "POR_PSD"],aliases=["Municipio: ","% Población sin derechohabiencia: "],  sticky=False, localize=True)
)
salud1.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=["NOMGEO", "% Personas con discapacidad motriz"],aliases=["Municipio: ","% Personas con discapacidad motríz: "],  sticky=False, localize=True)
)
salud2.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=["NOMGEO", "%Personas con alguna condición mental"],aliases=["Municipio: ","%Personas con alguna condición mental: "],  sticky=False, localize=True)
)



#Control de localización del usuario
'''lc = LocateControl(locatingMsg='Espera mientras se encuentra tu ubicación',
                   locate_options={'enableHighAccuracy': True, 'watch':True},
                   strings={"title": "Encuentra tu ubicación", "popup": "Te encuentras cerca de aquí"}).add_to(m)'''
buscador= Search(
    layer=psdh,
    geom_type='Polygon',
    position='topright',
    placeholder='Busqueda por municipio...',
    collapsed=False,
    search_zoom='13',
    search_label='NOMGEO'
).add_to(m)

m.add_child(folium.LatLngPopup())


layer_control = folium.LayerControl(collapsed=True, position="topright", title="Custom Title").add_to(m)
#plugins.MiniMap().add_to(m)
plugins.MeasureControl(position='toright').add_to(m)
plugins.Fullscreen(position='bottomleft').add_to(m)

m
m.save('visor_salud.html')
